<a href="https://colab.research.google.com/github/if001/rwkv.cpp/blob/master/rwkv_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setup

In [ ]:
!pip install transformers pytorch-lightning==1.9 deepspeed wandb ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.2/766.2 kB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 k

In [ ]:
#@title Google Drive Options { display-mode: "form" }
save_models_to_drive = True #@param {type:"boolean"}
drive_mount = '/content/drive' #@param {type:"string"}
output_dir = 'rwkv-v4-lora' #@param {type:"string"}
tuned_model_name = 'tuned_7B_raven' #@param {type:"string"}

import os
if save_models_to_drive:
    from google.colab import drive
    drive.mount(drive_mount)
    output_path = f"{drive_mount}/MyDrive/models/{output_dir}" if save_models_to_drive else f"/content/{output_dir}"
else:
    output_path = "/content"

tuned_model_dir = f"{output_path}/{tuned_model_name}"
os.makedirs(tuned_model_dir, exist_ok=True)
os.makedirs(f"{output_path}/base_models/", exist_ok=True)

print(f"Saving models to {output_path}")

Mounted at /content/drive
Saving models to /content/drive/MyDrive/models/rwkv-v4-lora


In [ ]:
%cd /content
# !git clone https://github.com/Blealtan/RWKV-LM-LoRA
!git clone https://github.com/if001/RWKV-LM-LoRA-ja.git
# repo_dir = "/content/RWKV-LM-LoRA/RWKV-v4neo"
repo_dir = "/content/RWKV-LM-LoRA-ja/RWKV-v4neo"
%cd $repo_dir

/content
Cloning into 'RWKV-LM-LoRA-ja'...
remote: Enumerating objects: 1773, done.
remote: Counting objects: 100% (1773/1773), done.
remote: Compressing objects: 100% (683/683), done.
remote: Total 1773 (delta 1117), reused 1744 (delta 1088), pack-reused 0
Receiving objects: 100% (1773/1773), 18.47 MiB | 10.47 MiB/s, done.
Resolving deltas: 100% (1117/1117), done.
/content/RWKV-LM-LoRA-ja/RWKV-v4neo


In [ ]:
%cd $repo_dir
!git pull

## load model

In [ ]:
#@title Base Model Options
#@markdown Using any of the listed options will download the checkpoint from huggingface
base_model_name = "RWKV-4-raven" #@param ["RWKV-4-Pile-14B", "RWKV-4-Pile-7B", "RWKV-4-Pile-3B", "RWKV-4-Pile-1B5", "RWKV-4-Pile-430M", "RWKV-4-Pile-169M", "RWKV-4-raven"]

if base_model_name == "RWKV-4-Pile-169M":
    base_model_file = "RWKV-4-Pile-169M-20220807-8023.pth"
    n_layer = 12
    n_embd = 768
elif base_model_name == "RWKV-4-Pile-430M":
    base_model_file = "RWKV-4-Pile-430M-20220808-8066.pth"
    n_layer = 24
    n_embd = 1024
elif base_model_name == "RWKV-4-Pile-1B5":
    base_model_file = "RWKV-4-Pile-1B5-20220903-8040.pth"
    n_layer = 24
    n_embd = 2048
elif base_model_name == "RWKV-4-Pile-3B":
    base_model_file = "RWKV-4-Pile-3B-20221008-8023.pth"
    n_layer = 32
    n_embd = 2560
elif base_model_name == "RWKV-4-Pile-7B":
    base_model_file = "RWKV-4-Pile-7B-20221115-8047.pth"
    n_layer = 32
    n_embd = 4096
elif base_model_name == "RWKV-4-Pile-14B":
    base_model_file = "RWKV-4-Pile-14B-20230213-8019.pth"
    n_layer = 40
    n_embd = 5120
elif base_model_name == "RWKV-4-raven":
    base_model_file = "RWKV-4-Raven-7B-v10-Eng89%25-Jpn10%25-Other1%25-20230420-ctx4096.pth"
    n_layer = 32
    n_embd = 4096

base_model_url = f"https://huggingface.co/BlinkDL/{base_model_name.lower()}/resolve/main/{base_model_file}"
base_model_path = f"{output_path}/base_models/{base_model_file}"

if save_models_to_drive and not os.path.exists(base_model_path):
    import urllib.request
    print(f"Downloading {base_model_name} this may take a while")
    urllib.request.urlretrieve(base_model_url, base_model_path)

print(f"Using {base_model_path} as base")

Using /content/drive/MyDrive/models/rwkv-v4-lora/base_models/RWKV-4-Raven-7B-v10-Eng89%25-Jpn10%25-Other1%25-20230420-ctx4096.pth as base


## train

In [ ]:
#@title Generate Training Data
#@markdown `input_file` should be the path to a single file that contains the text you want to fine-tune with.
#@markdown Either upload a file to this notebook instance or reference a file in your Google drive.
import numpy as np
from transformers import PreTrainedTokenizerFast

input_file = "/content/drive/MyDrive/t5_ranpo/dataset/databricks-dolly-15k-ja.json" #@param {type:"string"}
output_file = '/content/drive/MyDrive/t5_ranpo/dataset/databricks-dolly-15k-ja_rwkv_lora.npy'  #@param {type:"string"}


def gen_template(ins, inp, out):
    if inp == "":
        return f"""
        ### 指示:
        {ins}

        ### 応答:
        {out}
        """

    return f"""
    ### 指示:
    {ins}

    ### 入力:
    {inp}

    ### 応答:
    {out}
    """

def to_token(base_json, tokenizer):
    tokens = []
    for v in base_json:
        ins = v['instruction']
        inp = v['input']
        out = v['output']
        template = gen_template(ins, inp, out)
        token = tokenizer.encode(template)
        print(f'Tokenized length = {len(template)}')
        # print(token)
        tokens += token
    return tokens

import json
with open(input_file, encoding="utf-8") as f:
    j = json.load(f)

tokenizer = PreTrainedTokenizerFast(tokenizer_file=f'{repo_dir}/20B_tokenizer.json')

tokens = to_token(j, tokenizer)
out = np.array(tokens, dtype='uint16')
np.save(output_file, out, allow_pickle=False)
print(f'save... {output_file}')


KeyboardInterrupt: ignored

In [ ]:
#@title Begin Training with these Options { display-mode: "form" }
%cd $repo_dir

n_epoch = 5 #@param {type:"integer"}
epoch_save_frequency = 5 #@param {type:"integer"}
batch_size =  1 #@param {type:"integer"} 
ctx_len = 1024 #@param {type:"integer"}
vocab_size = 50277 #@param {type:"integer"}
train_file = "/content/drive/MyDrive/t5_ranpo/dataset/databricks-dolly-15k-ja_rwkv_lora.npy"   #@param {type:"string"}
data_type = 'numpy' #@param ['numpy', 'json']

!python3 train.py \
  --load_model $base_model_path \
  --proj_dir $tuned_model_dir \
  --data_file $train_file \
  --data_type "numpy" \
  --vocab_size $vocab_size \
  --ctx_len $ctx_len \
  --epoch_save $epoch_save_frequency \
  --epoch_count $n_epoch \
  --n_layer $n_layer \
  --n_embd $n_embd \
  --epoch_steps 1000 --epoch_begin 0  --micro_bsz $batch_size --pre_ffn 0 --head_qk 0 --lr_init 1e-5 --lr_final 1e-5 --warmup_steps 0 --beta1 0.9 --beta2 0.999 --adam_eps 1e-8 --accelerator gpu --devices 1 --precision bf16 --strategy deepspeed_stage_2 --grad_cp 0 \
  --lora --lora_r 8 --lora_alpha 32 --lora_dropout 0.01

/content/RWKV-LM-LoRA-ja/RWKV-v4neo
########## work in progress ##########

############################################################################
#
# RWKV-4 BF16 on 1x1 GPU, bsz 1x1x1=1, deepspeed_stage_2 
#
# Data = /content/drive/MyDrive/t5_ranpo/dataset/databricks-dolly-15k-ja_rwkv_lora.npy (numpy), ProjDir = /content/drive/MyDrive/models/rwkv-v4-lora/tuned_3B
#
# Epoch = 0 to 4 (will continue afterwards), save every 5 epoch
#
# Each "epoch" = 1000 steps, 1000 samples, 1024000 tokens
#
# Model = 32 n_layer, 2560 n_embd, 1024 ctx_len
# LoRA = enabled, 8 r, 32.0 alpha, 0.01 dropout, on att,ln,time
#
# Adam = lr 1e-05 to 1e-05, warmup 0 steps, beta (0.9, 0.999), eps 1e-08
#
# Found torch 2.0.0+cu118, recommend 1.13.1+cu117 or newer
# Found deepspeed 0.9.1, recommend 0.7.0 (faster than newer versions)
# Found pytorch_lightning 1.9.0, recommend 1.9.1 or newer
#
############################################################################

{'load_model': '/content/drive/MyDrive/mode

## predict

In [ ]:
#@title predict { display-mode: "form" }
load_model = '/content/drive/MyDrive/models/rwkv-v4-lora/base_models/RWKV-4-Pile-3B-20221008-8023'
ctx_len = 1024 #@param {type:"integer"}
vocab_size = 50277 #@param {type:"integer"}
run_device = 'cpu'  #@param {type:"string"} ['cpu', 'cuda']
float_mode='fp32'  #@param {type:"string"} ['fp16', 'fp32']

model_lora= '/content/drive/MyDrive/models/rwkv-v4-lora/tuned/rwkv-5'  #@param {type:"string"}
lora_r =8
lora_alpha = 32


%cd $repo_dir

!python run.py \
--load_model $load_model \
--ctx_len $ctx_len \
--n_embd $n_embd \
--vocab_size $vocab_size \
--run_device $run_device \
--float_mode $float_mode \
--model_lora $model_lora \
--lora_r $lora_r \
--lora_alpha $lora_alpha \
--context '### 指示: 日本で最も高い山を教えてください\n### 応答:'

/content/RWKV-LM-LoRA-ja/RWKV-v4neo

Using CPU. Loading /fsx/BlinkDL/HF-MODEL/rwkv-4-pile-7b/RWKV-4-Pile-7B-20221115-8047...

RWKV_HEAD_QK_DIM 0 RWKV_JIT_ON 1

merging blocks.13.att.key.lora_A and blocks.13.att.key.lora_B into blocks.13.att.key.weight
merging blocks.27.att.receptance.lora_A and blocks.27.att.receptance.lora_B into blocks.27.att.receptance.weight
merging blocks.15.att.key.lora_A and blocks.15.att.key.lora_B into blocks.15.att.key.weight
merging blocks.13.att.value.lora_A and blocks.13.att.value.lora_B into blocks.13.att.value.weight
merging blocks.8.att.key.lora_A and blocks.8.att.key.lora_B into blocks.8.att.key.weight
merging blocks.22.att.receptance.lora_A and blocks.22.att.receptance.lora_B into blocks.22.att.receptance.weight
merging blocks.4.att.value.lora_A and blocks.4.att.value.lora_B into blocks.4.att.value.weight
merging blocks.3.att.receptance.lora_A and blocks.3.att.receptance.lora_B into blocks.3.att.receptance.weight
merging blocks.2.att.key.lora_A and bl

## predict as langchain

In [ ]:
!pip install rwkv
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.5/626.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9


In [ ]:
#@title load model for predict(as pip) { display-mode: "form" }

from langchain.llms import RWKV
import os
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0'


# Load the model (supports full path, relative path, and remote paths)
# model_name = "/content/drive/MyDrive/models/rwkv-v4-lora/base_models/RWKV-4-Pile-7B-20221115-8047.pth"
model_name = "/content/drive/MyDrive/models/rwkv-v4-lora/base_models/RWKV-4-Raven-7B-v10-Eng89%25-Jpn10%25-Other1%25-20230420-ctx4096.pth"

gglm_file = 'jpn10%_gglm.bin'  #@param {"type": "string"}
model_name = f'{tuned_model_dir}/{gglm_file}'

strategy= 'cuda fp16' #@param ['cuda fp16', 'cpu fp32']
token_path = f'{repo_dir}/20B_tokenizer.json'


llm = RWKV(
    model=model_name,
    strategy=strategy, 
    tokens_path=token_path
)

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /content/drive/MyDrive/models/rwkv-v4-lora/tuned_7B_raven/jpn10%_gglm.bin.pth ...


FileNotFoundError: ignored

In [ ]:
#@title run predict(as pip) { display-mode: "form" }
instruction='北海道札幌市の人口を教えてください' #@param {type:"string"}

def gen(ins):
  return f"""
  ### 指示:
  {ins}
  
  ### 応答:
  """

prompt = gen(instruction)
output = llm(prompt, stop=['\n### 指示: ', '\n\t## 指示: '])

print(output)

北海道札幌市の人口は1,724,848人です。


## CPP to bin

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%cd /content
!git clone --recursive https://github.com/saharNooby/rwkv.cpp.git
cpp_repo='/content/rwkv.cpp'

/content
Cloning into 'rwkv.cpp'...
remote: Enumerating objects: 1547, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1547 (delta 142), reused 133 (delta 129), pack-reused 1377
Receiving objects: 100% (1547/1547), 5.16 MiB | 28.42 MiB/s, done.
Resolving deltas: 100% (942/942), done.
Submodule 'ggml' (https://github.com/saharNooby/ggml) registered for path 'ggml'
Cloning into '/content/rwkv.cpp/ggml'...
remote: Enumerating objects: 842, done.        
remote: Counting objects: 100% (516/516), done.        
remote: Compressing objects: 100% (170/170), done.        
remote: Total 842 (delta 362), reused 432 (delta 320), pack-reused 326        
Receiving objects: 100% (842/842), 3.52 MiB | 8.04 MiB/s, done.
Resolving deltas: 100% (510/510), done.
Submodule path 'ggml': checked out 'bfa8d5b5ab4ffbae4c5f97525c3890f38619056d'


In [ ]:
%cd $cpp_repo
!cmake -DBUILD_SHARED_LIBS=ON .
!cmake --build . --config Release

/content/rwkv.cpp
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Linux detected
-- Configuring done
-- Generating done
-- Build files have been written to: /content/rwkv.cpp
[  9%] Building C object ggml/src/CMakeFiles/ggml.dir/ggml.c.o
/c

In [ ]:
## gglm
gglm_file = f'{tuned_model_dir}/jpn10%_gglm.bin'


# !python rwkv/convert_pytorch_to_ggml.py $base_model_path  $gglm_file float16

## predict as cpp

In [ ]:
#@title load model for predict(as cpp) { display-mode: "form" }

%cd $cpp_repo'/rwkv'


import tokenizers
import torch

import rwkv_cpp_model
import rwkv_cpp_shared_library
import sampling

gglm_file = 'jpn10%_gglm.bin'  #@param {"type": "string"}
model_path = f'{tuned_model_dir}/{gglm_file}'
tokenizer_path = token_path = './20B_tokenizer.json'


MAX_GENERATION_LENGTH: int = 250
TEMPERATURE: float = 0.8
TOP_P: float = 0.5

library = rwkv_cpp_shared_library.load_rwkv_shared_library()
print(f'System info: {library.rwkv_get_system_info_string()}')
print('Loading RWKV model')
model = rwkv_cpp_model.RWKVModel(
    library,
    model_path
)
tokenizer = tokenizers.Tokenizer.from_file(str(tokenizer_path))

/content/rwkv.cpp/rwkv
System info: AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
Loading RWKV model


In [ ]:
#@title run predict(as cpp) { display-mode: "form" }
OUTPUT_LEN = 50

def gen(prompt):
    temperature = TEMPERATURE
    top_p = TOP_P

    prompt_tokens = tokenizer.encode(prompt).ids
    # prompt_token_count = len(prompt_tokens)
    logits_arr = []

    ## init
    logits, model_state = None, None
    for token in prompt_tokens:
      logits, model_state = model.eval(token, model_state, model_state, logits)

    ## new line
    output_tokens = []
    for i in range(OUTPUT_LEN):
      token = sampling.sample_logits(logits, temperature, top_p)
      logits, model_state = model.eval(token, model_state, model_state, logits)
      output_tokens.append(token)
      decoded = tokenizer.decode([token])
      #print('t', token)
      #print('d', decoded)
      #print('---')

    decoded = tokenizer.decode(output_tokens)
    print('output: ', decoded)

def gen_prompt(ins):
  return f"""
  ### 指示:
  {ins}
  
  ### 応答:
  """

instruct = '北海道札幌市の人口を教えてください' #@param {"type": "string"}
prompt = gen_prompt(instruct)
gen(prompt)

/content/rwkv.cpp/rwkv/rwkv_cpp_model.py:100: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  state_out.storage().data_ptr(),
/content/rwkv.cpp/rwkv/rwkv_cpp_model.py:101: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  logits_out.storage().data_ptr()
/content/rwkv.cpp/rwkv/rwkv_cpp_model.py:82: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directl

t 15899
d �
---
t 234
d �
---
t 21998
d �
---
t 117
d �
---
t 34260
d 道
---
t 5959
d �
---
t 244
d �
---
t 12462
d �
---
t 223
d �
---
t 42335
d 市
---
t 3917
d の
---
t 13484
d 人
---
t 44002
d 口
---
t 6418
d は
---
t 19
d 2
---
t 13
d ,
---
t 41923
d 076
---
t 13
d ,
---
t 24
d 7
---
t 1348
d 24
---
t 13484
d 人
---
t 20776
d です
---
t 4340
d 。
---
t 0
d 
---
t 30003
d Below
---
t 310
d  is
---
t 271
d  an
---
t 9775
d  instruction
---
t 326
d  that
---
t 8631
d  describes
---
t 247
d  a
---
t 4836
d  task
---
t 15
d .
---
t 19566
d  Write
---
t 247
d  a
---
t 2380
d  response
---
t 326
d  that
---
t 20420
d  appropriately
---
t 29141
d  completes
---
t 253
d  the
---
t 2748
d  request
---
t 15
d .
---
t 187
d 

---
t 187
d 

---
t 10548
d Inst
---
t 2705
d ruction
---
t 27
d :
---
t 187
d 

---
t 40731
d Generate
---
t 247
d  a
---
output:  北海道札幌市の人口は2,076,724人です。Below is an instruction that describes a task. Write a response that appropriately completes the request.

Instruction:
Generat